In [89]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os 

In [90]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [93]:
img_width = 50
img_height = 50
batch_size = 32

Dataset loading Using dataset_from_directory

In [ ]:
###   Using dataset_from_directory

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    'PetImages/',
    labels='inferred',
    label_mode ="categorical", #int, binary
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="training",    
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    'PetImages/',
    labels='inferred',
    label_mode ="categorical", #int, binary
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="validation",    
)

In [ ]:
print(ds_train)

Data Augmentation

In [ ]:
# Data Augmentation using functions

def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

def augment(image,label):
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
    image = tf.image.random_flip_left_right(image)  # 50%
    image = tf.image.random_flip_up_down(image) #%50%
    
    return image, label

 ## Data augmentation function calls
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE =32

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.map(augment,num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)


ds_validation = ds_validation.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_validation = ds_validation.batch(BATCH_SIZE)
ds_validation = ds_validation.prefetch(AUTOTUNE)


In [ ]:
model = keras.Sequential ([
    layers.Input((50,50,1)),
    layers.Conv2D(32,(3,3),padding = 'same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    
    layers.Conv2D(16,(3,3),padding = 'same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    
    layers.Flatten(),
    
    layers.Dropout(0.5),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation ='relu'),
    layers.Dropout(0.5),
    
    layers.Dense(2),
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_validation)

In [ ]:
model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_validation)

In [ ]:
## TF >= 2.3.0 (Sequential)

data_augmentation = keras.Sequential(
    [
        #layers.experimental.preprocessing.Resizing(height=32, width=32,),
        layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
        layers.experimental.preprocessing.RandomContrast(factor=0.1,),
    ]
)

## data_augmentation function calls when model is building
model = keras.Sequential ([
    layers.Input((50,50,1)),
    data_augmentation,
    layers.Conv2D(32,(3,3),padding = 'same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    
    layers.Conv2D(16,(3,3),padding = 'same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    
    layers.Flatten(),
    
    layers.Dropout(0.5),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation ='relu'),
    layers.Dropout(0.5),
    
    layers.Dense(2),
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_validation)

Dataset loading ImageDataGenerator and flow_from_directory

In [99]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=5,
    zoom_range=(0.95, 0.95),
    horizontal_flip=False,
    vertical_flip=False,
    data_format="channels_last",
    validation_split=0.0,
)

train_generator = datagen.flow_from_directory(
    "PetImages/",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="sparse",
    shuffle=True,
    subset="training",
    seed=123,
)

Found 25000 images belonging to 2 classes.


In [97]:
model = keras.Sequential ([
    layers.Input((50,50,1)),
    layers.Conv2D(32,(3,3),padding = 'same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    
    layers.Conv2D(16,(3,3),padding = 'same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    
    layers.Flatten(),
    
    layers.Dropout(0.5),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation ='relu'),
    layers.Dropout(0.5),
    
    layers.Dense(2),
])

In [100]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=True),],
    metrics=["accuracy"],
)


model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=25,
    verbose=2,
)

Epoch 1/10
25/25 - 19s - loss: 0.6976 - accuracy: 0.5263 - 19s/epoch - 741ms/step
Epoch 2/10
25/25 - 17s - loss: 0.6941 - accuracy: 0.4737 - 17s/epoch - 673ms/step
Epoch 3/10
25/25 - 16s - loss: 0.6935 - accuracy: 0.5063 - 16s/epoch - 624ms/step
Epoch 4/10
25/25 - 17s - loss: 0.6954 - accuracy: 0.5088 - 17s/epoch - 675ms/step
Epoch 5/10
25/25 - 15s - loss: 0.6944 - accuracy: 0.5000 - 15s/epoch - 600ms/step
Epoch 6/10
25/25 - 15s - loss: 0.6925 - accuracy: 0.5322 - 15s/epoch - 597ms/step
Epoch 7/10
25/25 - 14s - loss: 0.6936 - accuracy: 0.4938 - 14s/epoch - 578ms/step
Epoch 8/10
25/25 - 14s - loss: 0.6894 - accuracy: 0.5450 - 14s/epoch - 579ms/step
Epoch 9/10
25/25 - 13s - loss: 0.6848 - accuracy: 0.5600 - 13s/epoch - 522ms/step
Epoch 10/10
25/25 - 13s - loss: 0.6879 - accuracy: 0.5400 - 13s/epoch - 531ms/step
